In [7]:
from dotenv import load_dotenv
load_dotenv(override=True)

import os

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [8]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")

In [ ]:
from autogen_agentchat.messages import TextMessage
message = TextMessage(content="I want visit Turkey", source="user")
message

TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 9, 22, 9, 54, 44, 404992, tzinfo=datetime.timezone.utc), content='I want to play football', type='TextMessage')

In [13]:
from autogen_agentchat.agents import AssistantAgent

agent = AssistantAgent(
    name="travel_guide",
    model_client=model_client,
    system_message = "You are a helpful assistant for airline, you give short humorous answers!",
    model_client_stream=True
)

In [15]:
from autogen_core import CancellationToken
message = TextMessage(content="I want visit Turkey", source="user")
response = await agent.on_messages([message], cancellation_token=CancellationToken())
response.chat_message.content

'Great choice! Just make sure your passport is as valid as your desire for kebabs! 🍢✈️'

In [34]:
import sqlite3

if os.path.exists("flight_fares.db"):
    os.remove("flight_fares.db")

conn = sqlite3.connect("flight_fares.db")

c = conn.cursor()
c.execute("CREATE TABLE cities (city_name TEXT PRIMARY KEY, round_trip_price REAL)")

conn.commit()
conn.close()

In [35]:
# Populate our database
def save_city_price(city_name, round_trip_price):
    conn = sqlite3.connect("flight_fares.db")
    c = conn.cursor()
    c.execute("REPLACE INTO cities (city_name, round_trip_price) VALUES (?, ?)", (city_name.lower(), round_trip_price))
    conn.commit()
    conn.close()

# Some cities!
save_city_price("London", 299)
save_city_price("Paris", 399)
save_city_price("Rome", 499)
save_city_price("Madrid", 550)
save_city_price("Barcelona", 580)
save_city_price("Berlin", 525)

In [36]:
# Method to get price for a city
def get_city_price(city_name: str) -> float | None:
    """ Get the roundtrip ticket price to travel to the city """
    conn = sqlite3.connect("flight_fares.db")
    c = conn.cursor()
    c.execute("SELECT round_trip_price FROM cities WHERE city_name = ?", (city_name.lower(),))
    result = c.fetchone()
    conn.close()
    return result[0] if result else None

In [38]:
print(get_city_price("Rome"))

499.0


In [40]:
from autogen_agentchat.agents import AssistantAgent

smart_agent = AssistantAgent(
    name="smart_airline_agent",
    model_client=model_client,
    system_message="You are a helpful assistant for an airline. You give short, humorous answers, including the price of a roundtrip ticket.",
    model_client_stream=True,
    tools=[get_city_price],
    reflect_on_tool_use=True
)

In [42]:
message = TextMessage(content="I want visit Madrid", source="user")
response = await smart_agent.on_messages([message], cancellation_token=CancellationToken())
for inner_message in response.inner_messages:
    print(inner_message.content)
response.chat_message.content

[FunctionCall(id='call_Y0cQhiO43gUvFT07lH4S7DEt', arguments='{"city_name":"Madrid"}', name='get_city_price')]
[FunctionExecutionResult(content='550.0', name='get_city_price', call_id='call_Y0cQhiO43gUvFT07lH4S7DEt', is_error=False)]


'Fantastic choice! A roundtrip ticket to Madrid will run you about $550. You could almost buy a lifetime supply of churros with that! 🍩✈️'